In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 47.5 MB/s 
     |████████████████████████████████| 596 kB 5.8 MB/s 
     |████████████████████████████████| 86 kB 1.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup, GPT2Model
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/DataSet/동화 데이터.xlsx')
len(df)

245

In [ ]:
texts = []
for i in df['storytext']:
    texts.extend(i.split('\n'))

In [ ]:
texts = [x.strip() for x in texts if x != '']

In [ ]:
df = pd.DataFrame([x.strip() for x in texts],columns = ['text'])

In [ ]:
df

,text
0,"옛날 옛적, 가난해서 장가를 못간 나무꾼이 있었습니다."
1,하루는 나무를 하러 산에 갔다가 사냥꾼에게 쫓기는 노루를 발견하고 노루를 숨겨주었습니다.
2,"노루는 보은의 의미로 선녀탕을 알려주고, 옷을 숨기면 혼인을 할 수 있다고 알려주고..."
3,나무꾼은 노루가 알려준 대로 선녀탕을 찾아가 옷을 숨기고 막내선녀와 결혼을 하게 되...
4,하지만 하늘이 그리웠던 선녀는 나무꾼에게 날개옷을 한 번만 보여달라며 사정을 하고 ...
...,...
12176,‘이제 열쇠만 맞으면!’라고 소년이 생각했지요. ‘이 예쁜 상자 안에 든 귀중품들을...
12177,하지만 소년이 아무리 살펴보아도 거기엔 열쇠구멍이 없지 뭐예요. 그러다 결국은 열쇠...
12178,소년이 끼워보니 열쇠가 열쇠구멍에 딱 들어맞았어요.
12179,그런 다음 소년은 열쇠를 한 번 찰칵 하고 돌려보았어요.


In [ ]:
# 데이터셋 인코딩
class Create_data(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="skt/kogpt2-base-v2", max_length=1024):

        self.tokenizer = PreTrainedTokenizerFast.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['text']:
            self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))

        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]
    
dataset = Create_data(df['text'], truncate=True, gpt2_type="skt/kogpt2-base-v2") 

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
#Get the tokenizer and model
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 
model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=32, epochs=10, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="skt/kogpt2-base-v2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [ ]:
model = train(dataset, model, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Training epoch 0
0


12185it [07:58, 25.44it/s]


Training epoch 1
tensor(0.0982, device='cuda:0', grad_fn=<NllLossBackward0>)


12185it [07:59, 25.43it/s]


...

12185it [07:59, 25.42it/s]


Training epoch 8
tensor(0.5409, device='cuda:0', grad_fn=<NllLossBackward0>)


12185it [07:59, 25.43it/s]


Training epoch 9
tensor(0.4549, device='cuda:0', grad_fn=<NllLossBackward0>)


12185it [07:59, 25.43it/s]


In [ ]:
# 토크나이저 저장
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/kogpt2_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# 모델 저장
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/kogpt_model.pt')

In [ ]:
# 토크나이저 불러오기
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/kogpt2_tokenizer.pickle', 'rb') as handle:
      tokenizer = pickle.load(handle)

In [ ]:
# 모델 불러오기
model = torch.load('/content/drive/MyDrive/Colab Notebooks/kogpt_model2.pt')

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=70, #maximum number of words
    top_p=0.92,
    top_k=50,
    temperature=0.9,
    repetition_penalty=1.5
):

    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

In [ ]:
import re
import random
import torch.backends.cudnn as cudnn

# 시드 고정
# torch.manual_seed(0)
# torch.cuda.manual_seed(0)
# torch.cuda.manual_seed_all(0)
# np.random.seed(0)
# cudnn.benchmark = False
# cudnn.deterministic = True
# random.seed(0)

def generate_text(text):
    res = generate(model.to('cpu'), tokenizer, text, entry_count=1)[0]
    to_remove = res.split('.')[-1]
    res = res.replace(to_remove,'')
    print()
    return res

In [ ]:
prompt = '그때 어디선가 발소리가 들렸어요.'
print(generate_text(prompt))

100%|██████████| 1/1 [00:17<00:00, 17.77s/it]


그때 어디선가 발소리가 들렸어요. "어디서 그 낯선 교회까지?" 그리고는 곧장 그 신부가 이목을 끌려고 몸을 기울였어요. 그때, 그녀는 아주 어린 여자아이였어요. 소녀에게 따라나섰던 건 아니었지만, 그럼에도 불구하고 그녀는 무척 명랑해서 아무 말도 할 수 없었어요.


In [ ]:
print(tokenizer.tokenize("독수리 부리는 왜 노랄까?"))

['▁독', '수리', '▁부리는', '▁왜', '▁노', '랄', '까?']


In [ ]:
print(tokenizer.encode("독수리 부리는 왜 노랄까?"))

[9229, 16721, 30628, 10401, 9153, 7375, 47804]
